In [1]:
import tensorflow.keras.backend as K
import tensorflow
import tensorflow as tf
import math
import gc
import numpy as np
import os

jobs = int(os.cpu_count()/2)

print(np.__version__)
import time, math
from tqdm import tqdm_notebook as tqdm

from joblib import Parallel, delayed
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import copy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
from importlib import reload

import pandas as pd
import fastnet as fn
from fastnet.model.utils import ConvBn2D
from fastnet.misc import msg

from tensorflow.keras.backend import clear_session

import logging

logger = logging.getLogger("App")
logger.setLevel(logging.DEBUG)



1.16.2


In [2]:
random_pad_crop = fn.get_first_argument_transformer(fn.get_random_pad_crop(4,4,32,32))
cutout = fn.get_first_argument_transformer(fn.get_cutout_eraser(0,255))
hflip = fn.get_first_argument_transformer(fn.get_hflip_aug())

transformations = fn.combine_transformers(random_pad_crop,hflip,cutout)

## Using Sub-Classed Keras Model

In [9]:
batch_size = BATCH_SIZE = 512
EPOCHS = 5

train,test = fn.get_cifar10("cifar10",batch_size)

train = train.map(transformations).map(lambda x,y: (x/255.0,y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test = test.map(lambda x,y: (x/255.0,y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [4]:

class SimpleCifarNet(tf.keras.Model):
    def __init__(self, start_kernels=32, weight=0.125, **kwargs):
        super().__init__(**kwargs)
        c = start_kernels
        self.max_pool = tf.keras.layers.MaxPooling2D()
        self.init_conv_bn = ConvBn2D(c, kernel_size=3)
        self.c0 = ConvBn2D(c, kernel_size=3)
        
        self.c1 = ConvBn2D(c*2, kernel_size=3)
        self.c2 = ConvBn2D(c*2, kernel_size=3)
        
        self.c3 = ConvBn2D(c*4, kernel_size=3)
        self.c4 = ConvBn2D(c*4, kernel_size=3)
        

        self.pool = tf.keras.layers.GlobalMaxPool2D()
        self.linear = tf.keras.layers.Dense(10, kernel_initializer='glorot_uniform', use_bias=False)
        self.weight = weight
        
    def call(self, x):
        
        h = self.max_pool(self.c0(self.init_conv_bn(x)))
        h = self.max_pool(self.c2(self.c1(h)))
        h = self.max_pool(self.c4(self.c3(h)))
        h = self.pool(h)
        h = self.linear(h) * self.weight
        return h

In [5]:
model = SimpleCifarNet(start_kernels=8,dynamic=True)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=loss,metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)])

callbacks=[]

model.fit(train, epochs=EPOCHS, callbacks=callbacks,verbose=1)

Epoch 1/5
98/98 [==============================] - 87s 884ms/step - loss: 1.8586 - sparse_categorical_accuracy: 0.2502 - sparse_top_k_categorical_accuracy: 0.5665
Epoch 2/5
98/98 [==============================] - 86s 873ms/step - loss: 1.5127 - sparse_categorical_accuracy: 0.4209 - sparse_top_k_categorical_accuracy: 0.7726
Epoch 3/5
98/98 [==============================] - 84s 860ms/step - loss: 1.3675 - sparse_categorical_accuracy: 0.4917 - sparse_top_k_categorical_accuracy: 0.8138
Epoch 4/5
98/98 [==============================] - 85s 868ms/step - loss: 1.2692 - sparse_categorical_accuracy: 0.5310 - sparse_top_k_categorical_accuracy: 0.8423
Epoch 5/5
98/98 [==============================] - 87s 887ms/step - loss: 1.1845 - sparse_categorical_accuracy: 0.5652 - sparse_top_k_categorical_accuracy: 0.8593


In [7]:
res = model.evaluate(test)
print("\nLoss = %.2f, Acc = %.2f, Top 3 Acc = %.2f"%tuple(res))

     20/Unknown - 4s 180ms/step - loss: 1.7010 - sparse_categorical_accuracy: 0.4326 - sparse_top_k_categorical_accuracy: 0.7531
Loss = 1.70, Acc = 0.43, Top 3 Acc = 0.75


In [7]:
train,test = fn.get_cifar10("cifar10",batch_size)

train = train.map(transformations).map(lambda x,y: (x/255.0,y))
test = test.map(lambda x,y: (x/255.0,y))
for x,y in train.take(1):
    tf.print(tf.shape(x),tf.shape(y))
    tf.print(x.dtype,y.dtype)

[512 32 32 3] [512 1]
tf.float32 tf.int32


## Using Training Loop with One Cycle LR

In [3]:
class SimpleCifarLoopNet(tf.keras.Model):
    def __init__(self, start_kernels=32, weight=0.125, **kwargs):
        super().__init__(**kwargs)
        c = start_kernels
        self.max_pool = tf.keras.layers.MaxPooling2D()
        self.init_conv_bn = ConvBn2D(c, kernel_size=3)
        self.c0 = ConvBn2D(c, kernel_size=3)
        
        self.c1 = ConvBn2D(c*2, kernel_size=3)
        self.c2 = ConvBn2D(c*2, kernel_size=3)
        
        self.c3 = ConvBn2D(c*4, kernel_size=3)
        self.c4 = ConvBn2D(c*4, kernel_size=3)
        

        self.pool = tf.keras.layers.GlobalMaxPool2D()
        self.linear = tf.keras.layers.Dense(10, kernel_initializer='glorot_uniform', use_bias=False)
        self.weight = weight
        
    def call(self, x, y):
        
        h = self.max_pool(self.c0(self.init_conv_bn(x)))
        h = self.max_pool(self.c2(self.c1(h)))
        h = self.max_pool(self.c4(self.c3(h)))
        h = self.pool(h)
        h = self.linear(h) * self.weight
        
        y = tf.reshape(y,shape=[tf.shape(y)[0]])
        ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
        loss = tf.reduce_sum(ce)
        correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
        return loss, correct

In [4]:
batch_size = BATCH_SIZE = 512
EPOCHS = 5
len_train = 50000
len_test = 10000
batches_per_epoch = len_train//BATCH_SIZE + 1
WEIGHT_DECAY = 1e-4
mid_epoch = 2
pre_end_epoch = 3


global_step = {"batch":0}
momentum_schedule =  lambda t: np.interp([t], [0, mid_epoch, pre_end_epoch, EPOCHS], [0.9, 0.8, 0.9, 0.9])[0] 
momentum_func = lambda: momentum_schedule(global_step["batch"]/batches_per_epoch)

lr_schedule = lambda t: np.interp([t], [0, mid_epoch, pre_end_epoch, EPOCHS], [0.04, 0.4, 0.04, 0.01])[0] # LR = 0.75
lr_func = lambda: lr_schedule(global_step["batch"]/batches_per_epoch)/BATCH_SIZE

opt = tf.keras.optimizers.SGD(learning_rate=lr_func, momentum=momentum_func, nesterov=True)
train_accs = []
test_accs = []

model = SimpleCifarLoopNet(start_kernels=8,dynamic=True)

t = time.time()
for epoch in range(EPOCHS):
    tf.keras.backend.set_learning_phase(1)    
    train_loss = test_loss = train_acc = test_acc = 0.0
    train,test = fn.get_cifar10("cifar10",batch_size)
    train = train.map(transformations).map(lambda x,y: (x/255.0,y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    test = test.map(lambda x,y: (x/255.0,y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    for (x, y) in train:
        
        with tf.GradientTape() as tape:
            loss, correct = model(x, y)

        var = model.trainable_variables
        grads = tape.gradient(loss, var)
        for g, v in zip(grads, var):
            g += v * WEIGHT_DECAY * BATCH_SIZE
        _ = opt.apply_gradients(zip(grads, var))
        global_step["batch"] = global_step["batch"]+1
        train_loss += loss.numpy()
        train_acc += correct.numpy()
        train_accs.append(train_acc / len_train)
        
    tf.keras.backend.set_learning_phase(0)
    for (x, y) in test:
        loss, correct = model(x, y)
        test_loss += loss.numpy()
        test_acc += correct.numpy()
    test_accs.append(test_acc / len_test)
    logger.debug(msg("epoch = %2s"%epoch,'||train=> loss: %.3f' %(train_loss / len_train), 'acc: %.3f' % (train_acc / len_train), '||val=> loss: %.3f' % (test_loss / len_test), 'val acc: %.3f' %(test_acc / len_test), '%.1fs'%(time.time() - t)))
    
    

time_spent = time.time() - t
logger.debug(msg("Train acc = ",train_accs[-1], "Test acc =",test_accs[-1], "Time Taken = ", time_spent))


I0911 16:16:56.061398 140735639765888 <ipython-input-4-c21831f5303e>:52]  epoch =  0 ||train=> loss: 1.991 acc: 0.265 ||val=> loss: 2.259 val acc: 0.215 103.5s
I0911 16:18:26.425060 140735639765888 <ipython-input-4-c21831f5303e>:52]  epoch =  1 ||train=> loss: 1.539 acc: 0.434 ||val=> loss: 1.679 val acc: 0.416 193.8s
I0911 16:20:09.369811 140735639765888 <ipython-input-4-c21831f5303e>:52]  epoch =  2 ||train=> loss: 1.289 acc: 0.532 ||val=> loss: 1.099 val acc: 0.600 296.8s
I0911 16:21:45.334099 140735639765888 <ipython-input-4-c21831f5303e>:52]  epoch =  3 ||train=> loss: 1.183 acc: 0.574 ||val=> loss: 1.052 val acc: 0.620 392.7s
I0911 16:23:17.439698 140735639765888 <ipython-input-4-c21831f5303e>:52]  epoch =  4 ||train=> loss: 1.136 acc: 0.589 ||val=> loss: 1.008 val acc: 0.636 484.8s
I0911 16:23:17.440636 140735639765888 <ipython-input-4-c21831f5303e>:57]  Train acc =  0.5894 Test acc = 0.6365 Time Taken =  484.83531522750854


## Visualizing Images and Results (Coming Soon)

In [ ]:
# Particular Class misclassified as Particular class
# Particular Class misclassified as Any class
# Any Class misclassified as Particular class
# Any Class misclassified as Any class
# Confusion Matrix and Common misclassifications

# Top Confusions of Particular Actual Class
# Top Confusions of Particular Predicted Class